In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns


In [23]:
# Load Data
fixations_path = "/Volumes/TwoTeras/2_DataSets_Experiments_1_2/BehavioralData_Fixations_Wide.csv"
HumanA_Fixations = pd.read_csv(fixations_path, sep=",")

In [25]:
HumanA_Fixations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16538 entries, 0 to 16537
Data columns (total 25 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0.2                   16538 non-null  int64  
 1   Unnamed: 0.1                   16538 non-null  int64  
 2   Unnamed: 0                     16538 non-null  int64  
 3   SubjectID                      16538 non-null  int64  
 4   AbsolutError                   16538 non-null  float64
 5   SignedAngle-+180               16538 non-null  float64
 6   IQR                            16538 non-null  float64
 7   RT                             16538 non-null  float64
 8   DistanceToParticipant          16538 non-null  float64
 9   PointingTaskStartingLocations  16538 non-null  int64  
 10  TrialNumber                    16538 non-null  int64  
 11  StartPointID                   16538 non-null  int64  
 12  ID_for_StartingPosition        16538 non-null 

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

# Load Data
fixations_path = "/Volumes/TwoTeras/2_DataSets_Experiments_1_2/BehavioralData_Fixations_Wide.csv"
HumanA_Fixations = pd.read_csv(fixations_path, sep=",")

# Debugging: Check for missing values in key columns before transformation
print("Missing values before transformation:")
print(HumanA_Fixations[["Context", "AvatarPresenceCategory", "Agent_Category", "Experiment"]].isna().sum())

# Recoding variables, ensuring we handle NaNs correctly
HumanA_Fixations["ContextEffect"] = HumanA_Fixations["Context"].map({False: -0.5, True: 0.5})
HumanA_Fixations["Agent_Action_level"] = HumanA_Fixations["Agent_Category"].map({"Passive": -0.5, "Active": 0.5})

# Encoding "Match" based on Experiment and ContextEffect
HumanA_Fixations["Match"] = np.where(
    (HumanA_Fixations["Experiment"] == 1) & (HumanA_Fixations["ContextEffect"] == 0.5), 
    0.5, 
    np.where(
        (HumanA_Fixations["Experiment"] == 2) & (HumanA_Fixations["Agent_Action_level"] == 0.5), 
        -0.5, 
        0  # Default case
    )
)


# Recode Experiment variable
HumanA_Fixations["Experiment"] = HumanA_Fixations["Experiment"].map({1: -0.5, 2: 0.5})

# Drop rows with missing values before assigning categorical variables
HumanA_Fixations = HumanA_Fixations.dropna(subset=["ContextEffect",  "Agent_Action_level", "Match", "Experiment"])

# Convert numeric variables to categorical explicitly
HumanA_Fixations["ContextEffectf"] = pd.Categorical(
    HumanA_Fixations["ContextEffect"].map({-0.5: 'Residential', 0.5: 'Public'}),
    categories=['Residential', 'Public']
)


HumanA_Fixations["Agent_Action_levelf"] = pd.Categorical(
    HumanA_Fixations["Agent_Action_level"].map({-0.5: 'Passive', 0.5: 'Active'}),
    categories=['Passive', 'Active']
)
HumanA_Fixations["Matchf"] = pd.Categorical(
    HumanA_Fixations["Match"].map({-0.5: 'Not Congruent', 0.5: 'Congruent'}),
    categories=['Not Congruent', 'Congruent']
)
HumanA_Fixations["Experimentf"] = pd.Categorical(
    HumanA_Fixations["Experiment"].map({-0.5: 'Experiment 1', 0.5: 'Experiment 2'}),
    categories=['Experiment 1', 'Experiment 2']
)

# Ensure SubjectID is treated as a categorical variable
HumanA_Fixations["SubjectID"] = HumanA_Fixations["SubjectID"].astype("category")

# Center dwelling time variables
HumanA_Fixations["Dwelling_Time_Building_Gaze_Centered"] = (
    HumanA_Fixations["Dwelling_Time_Building_Gaze"] - HumanA_Fixations["Dwelling_Time_Building_Gaze"].mean()
)
HumanA_Fixations["Dwelling_Time_Agent_Gaze_Centered"] = (
    HumanA_Fixations["Dwelling_Time_Agent_Gaze"] - HumanA_Fixations["Dwelling_Time_Agent_Gaze"].mean()
)

# Drop rows with missi
HumanA_Fixations.info()

Missing values before transformation:
Context                   0
AvatarPresenceCategory    0
Agent_Category            0
Experiment                0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16538 entries, 0 to 16537
Data columns (total 33 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   Unnamed: 0.2                          16538 non-null  int64   
 1   Unnamed: 0.1                          16538 non-null  int64   
 2   Unnamed: 0                            16538 non-null  int64   
 3   SubjectID                             16538 non-null  category
 4   AbsolutError                          16538 non-null  float64 
 5   SignedAngle-+180                      16538 non-null  float64 
 6   IQR                                   16538 non-null  float64 
 7   RT                                    16538 non-null  float64 
 8   DistanceToParticipant                 16538

In [3]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

# Load Data
fixations_path = "/Volumes/TwoTeras/2_DataSets_Experiments_1_2/BehavioralData_Fixations_Wide.csv"
HumanA_Fixations = pd.read_csv(fixations_path, sep=",")

# Convert 'Context' explicitly to string first to avoid boolean interpretation issues
HumanA_Fixations["Context"] = HumanA_Fixations["Context"].astype(str)

# Recoding variables properly
HumanA_Fixations["ContextEffect"] = HumanA_Fixations["Context"].map({"False": -0.5, "True": 0.5})
HumanA_Fixations["AgentPresence"] = HumanA_Fixations["AvatarPresenceCategory"].map({"Omitted": -0.5, "Present": 0.5})
HumanA_Fixations["Agent_Action_level"] = HumanA_Fixations["Agent_Category"].map({"Passive": -0.5, "Active": 0.5})

# Encoding "Match" based on Experiment and ContextEffect
HumanA_Fixations["Match"] = np.where(
    (HumanA_Fixations["Experiment"] == 1) & (HumanA_Fixations["ContextEffect"] == 0.5), 
    0.5, 
    -0.5
)

# Recode Experiment variable
HumanA_Fixations["Experiment"] = HumanA_Fixations["Experiment"].map({1: -0.5, 2: 0.5})

# Drop rows with missing values BEFORE assigning categories
HumanA_Fixations = HumanA_Fixations.dropna(subset=["ContextEffect", "AgentPresence", "Agent_Action_level", "Match", "Experiment", "SubjectID"])

# Convert variables to categorical explicitly
HumanA_Fixations["ContextEffectf"] = pd.Categorical(
    HumanA_Fixations["ContextEffect"].map({-0.5: 'Residential', 0.5: 'Public'}),
    categories=['Residential', 'Public'],
    ordered=True
)
HumanA_Fixations["AgentPresencef"] = pd.Categorical(
    HumanA_Fixations["AgentPresence"].map({-0.5: 'Omitted', 0.5: 'Displayed'}),
    categories=['Omitted', 'Displayed'],
    ordered=True
)
HumanA_Fixations["Agent_Action_levelf"] = pd.Categorical(
    HumanA_Fixations["Agent_Action_level"].map({-0.5: 'Passive', 0.5: 'Active'}),
    categories=['Passive', 'Active'],
    ordered=True
)
HumanA_Fixations["Matchf"] = pd.Categorical(
    HumanA_Fixations["Match"].map({-0.5: 'Not Congruent', 0.5: 'Congruent'}),
    categories=['Not Congruent', 'Congruent'],
    ordered=True
)
HumanA_Fixations["Experimentf"] = pd.Categorical(
    HumanA_Fixations["Experiment"].map({-0.5: 'Experiment 1', 0.5: 'Experiment 2'}),
    categories=['Experiment 1', 'Experiment 2'],
    ordered=True
)

# Ensure SubjectID is categorical
HumanA_Fixations["SubjectID"] = HumanA_Fixations["SubjectID"].astype("category")

# Reset index after cleaning
HumanA_Fixations = HumanA_Fixations.reset_index(drop=True)

# Center dwelling time variables
HumanA_Fixations["Dwelling_Time_Building_Gaze_Centered"] = (
    HumanA_Fixations["Dwelling_Time_Building_Gaze"] - HumanA_Fixations["Dwelling_Time_Building_Gaze"].mean()
)
HumanA_Fixations["Dwelling_Time_Agent_Gaze_Centered"] = (
    HumanA_Fixations["Dwelling_Time_Agent_Gaze"] - HumanA_Fixations["Dwelling_Time_Agent_Gaze"].mean()
)

# Drop any remaining missing values
filtered_data = HumanA_Fixations.dropna().reset_index(drop=True)

# Debugging: Check the unique categories again
print("ContextEffectf categories:", filtered_data["ContextEffectf"].unique())
print("Agent_Action_levelf categories:", filtered_data["Agent_Action_levelf"].unique())
print("Matchf categories:", filtered_data["Matchf"].unique())

# Fit the mixed-effects model with two random effects (SubjectID & ID_for_StartingPosition)
full_model = smf.mixedlm(
    "AbsolutError ~ Dwelling_Time_Building_Gaze_Centered + Dwelling_Time_Agent_Gaze_Centered + "
    "C(ContextEffectf) * C(Agent_Action_levelf) + C(Matchf)",
    data=filtered_data,
    groups=filtered_data["SubjectID"],  # First random effect: SubjectID
    re_formula="1",  # Random intercept model
    vc_formula={"ID_for_StartingPosition": "0 + C(ID_for_StartingPosition)"}  # Second random effect
).fit(method="lbfgs")

# Print model summary
print(full_model.summary())


ContextEffectf categories: ['Residential', 'Public']
Categories (2, object): ['Residential' < 'Public']
Agent_Action_levelf categories: ['Passive', 'Active']
Categories (2, object): ['Passive' < 'Active']
Matchf categories: ['Not Congruent', 'Congruent']
Categories (2, object): ['Not Congruent' < 'Congruent']
                                  Mixed Linear Model Regression Results
Model:                             MixedLM                Dependent Variable:                AbsolutError
No. Observations:                  14295                  Method:                            REML        
No. Groups:                        57                     Scale:                             1753.3380   
Min. group size:                   160                    Log-Likelihood:                    -73829.9723 
Max. group size:                   327                    Converged:                         Yes         
Mean group size:                   250.8                                               

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
from scipy.stats import norm

# Set seed for reproducibility
np.random.seed(42)

# Extract relevant sample size information
num_subjects = filtered_data["SubjectID"].nunique()  # Number of unique subjects
num_obs = len(filtered_data)  # Total observations

# Define effect sizes to test (small → large)
effect_sizes = np.linspace(0, 0.3, 20)  # Change 0.5 if you expect stronger effects

# Store results
power_results = []

# Loop over effect sizes
for effect in effect_sizes:
    significant_count = 0
    simulations = 1000  # Number of bootstrap iterations
    
    for _ in range(simulations):
        # Simulate response variable with given effect size
        simulated_data = filtered_data.copy()
        simulated_data["AbsolutError"] += np.random.normal(loc=effect, scale=simulated_data["AbsolutError"].std(), size=num_obs)

        # Fit model
        model = smf.mixedlm(
            "AbsolutError ~ Dwelling_Time_Building_Gaze_Centered + Dwelling_Time_Agent_Gaze_Centered + "
            "C(ContextEffectf) * C(Agent_Action_levelf) + C(Matchf)",
            data=simulated_data,
            groups=simulated_data["SubjectID"]
        ).fit(method="lbfgs")

        # Check if effect is statistically significant (p < 0.05)
        p_value = model.pvalues.get("C(ContextEffectf)[T.Public]", 1)  # Choose key predictor
        if p_value < 0.05:
            significant_count += 1

    # Compute power (proportion of significant results)
    power = significant_count / simulations
    power_results.append((effect, power))

# Convert results to DataFrame
power_df = pd.DataFrame(power_results, columns=["Effect Size", "Power"])

# Find smallest effect size reaching 95% power
min_effect_95 = power_df[power_df["Power"] >= 0.95]["Effect Size"].min()

# Show power curve
import matplotlib.pyplot as plt

plt.plot(power_df["Effect Size"], power_df["Power"], marker="o", linestyle="-")
plt.axhline(0.95, color="r", linestyle="--", label="95% Power Threshold")
plt.xlabel("Effect Size")
plt.ylabel("Power")
plt.title("Sensitivity Analysis: Detectable Effect Size")
plt.legend()
plt.grid(True)
plt.show()

print(f"🔹 Smallest detectable effect size at 95% power: {min_effect_95:.3f}")


/opt/anaconda3/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:1634: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
/opt/anaconda3/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2054: UserWarning: The random effects covariance matrix is singular.
  warnings.warn(_warn_cov_sing)
/opt/anaconda3/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/opt/anaconda3/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2245: UserWarning: The random effects covariance matrix is singular.
  warnings.warn(_warn_cov_sing)
/opt/anaconda3/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
/opt/an

In [5]:

df = HumanA_Fixations
# Define the full mixed-effects model
full_model = smf.mixedlm("AbsolutError ~ Dwelling_Time_Building_Gaze + Dwelling_Time_Agent_Gaze + "
                         "ContextEffectf * Agent_Action_levelf + Matchf",
                         data=df, groups=df["SubjectID"]).fit(method="lbfgs")

print(full_model.summary())

# ------------------------
# Sensitivity Analysis
# ------------------------

# Define function to simulate power for different effect sizes
def sensitivity_analysis(sample_size, effect_sizes, num_simulations=500):
    power_results = []

    for effect in effect_sizes:
        significant_results = 0

        for _ in range(num_simulations):
            # Simulate random predictors
            Dwelling_Time_Building_Gaze = np.random.normal(0, 1, sample_size)
            Dwelling_Time_Agent_Gaze = np.random.normal(0, 1, sample_size)
            ContextEffect = np.random.choice([0, 1], sample_size)
            Agent_Action = np.random.choice([0, 1], sample_size)
            Match = np.random.choice([0, 1], sample_size)
            SubjectID = np.random.choice(range(1, 43), sample_size)

            # Generate dependent variable with varying effect sizes
            AbsolutError = (effect * Dwelling_Time_Building_Gaze +
                            effect * Dwelling_Time_Agent_Gaze +
                            effect * ContextEffect +
                            effect * Agent_Action +
                            effect * Match +
                            np.random.normal(0, 1, sample_size))

            sim_df = pd.DataFrame({
                "AbsolutError": AbsolutError,
                "SubjectID": SubjectID.astype(str),
                "Dwelling_Time_Building_Gaze": Dwelling_Time_Building_Gaze,
                "Dwelling_Time_Agent_Gaze": Dwelling_Time_Agent_Gaze,
                "ContextEffect": ContextEffect,
                "Agent_Action": Agent_Action,
                "Match": Match
            })

            # Fit mixed model for each simulation
            try:
                sim_model = smf.mixedlm("AbsolutError ~ Dwelling_Time_Building_Gaze + Dwelling_Time_Agent_Gaze + "
                                        "ContextEffect + Agent_Action + Match",
                                        data=sim_df, groups=sim_df["SubjectID"]).fit(method="lbfgs")

                # Count significant effects
                if (sim_model.pvalues < 0.05).any():
                    significant_results += 1
            except:
                continue  # Skip iterations where model fails to converge

        # Calculate power
        power = significant_results / num_simulations
        power_results.append({"Effect_Size": effect, "Power": power})

    return pd.DataFrame(power_results)

# Run Sensitivity Analysis for Effect Sizes from 0.01 to 0.5
effect_sizes = np.arange(0.01, 0.51, 0.05)
sample_size = df.shape[0]  # Use actual sample size from the dataset

power_df = sensitivity_analysis(sample_size, effect_sizes)

# Plot Sensitivity Analysis Results
plt.figure(figsize=(8, 5))
sns.lineplot(x=power_df["Effect_Size"], y=power_df["Power"], marker="o")
plt.axhline(0.80, color="r", linestyle="--", label="80% Power Threshold")
plt.xlabel("Effect Size (β)")
plt.ylabel("Power")
plt.title("Sensitivity Analysis: Detectable Effect Sizes")
plt.legend()
plt.show()

ValueError: negative dimensions are not allowed